table of contents  
1. Prepare  
2. Dataset Read, and generate movie

# Prepare

## Import Library

In [1]:
import os, sys
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation as animation
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

## Define Static Parameter

In [2]:
base_dir = os.path.join('..')
out_dir = os.path.join(base_dir, 'reports', 'movie', 'original')

sys.path.append(os.path.join(base_dir, 'scripts'))
import Metrica_Viz as mviz
import Metrica_PitchControl as mpc

# Dataset Reading, and Preprocess

In [3]:
data_dir = os.path.join(base_dir, 'datasets', 'preprocessed')
play_list = [play for play in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, play))]
print(play_list)

['Fulham 0 - [1] Liverpool', 'Genk 0 - [3] Liverpool', 'Bayern 0 - [1] Liverpool', 'Southampton 1 - [2] Liverpool', 'Bournemouth 0 - 3 Liverpool', 'Liverpool [1] - 0 Everton', 'Liverpool [3] - 0 Bournemouth', 'Liverpool [1] - 0 Wolves', 'Liverpool [2] - 1 Chelsea', 'Liverpool [3] - 0 Norwich', 'Liverpool [2] - 0 Porto', 'Liverpool [2] - 0 Everton', 'Liverpool [2] - 1 Newcastle', 'Liverpool [2] - 0 Salzburg', 'Liverpool [2] - 0 Man City', 'Liverpool [1] - 0 Watford', 'Leicester 0 - [3] Liverpool', 'Liverpool [4] - 0 Barcelona', 'Porto 0 - [2] Liverpool']


# generate movie(original)

In [4]:
for play in play_list:
    print(play)
    data_dir_tmp = os.path.join(data_dir, play)
    df_dict, color_dict = {}, {}
    for infile in os.listdir(data_dir_tmp):
        if not infile.startswith('events.'):
            team, color = infile.replace('.csv','').split('_')
            df = pd.read_csv(os.path.join(data_dir_tmp, infile), index_col=[0])
            df.columns = [c if c in ['Time [s]', 'ball_x', 'ball_y'] else f'{team}_{c}' for c in df.columns]
            df_dict[team] = df

            color_dict[team] = color
    
    mviz.save_match_clip(df_dict, fpath=out_dir, fname=play, figax=mviz.plot_pitch(field_color='twitter_dark_mode'), frames_per_second=20, team_color_dict=color_dict, annotate=True, include_player_velocities=True)

Fulham 0 - [1] Liverpool
Generating movie...done
Genk 0 - [3] Liverpool
Generating movie...done
Bayern 0 - [1] Liverpool
Generating movie...done
Southampton 1 - [2] Liverpool
Generating movie...done
Bournemouth 0 - 3 Liverpool
Generating movie...done
Liverpool [1] - 0 Everton
Generating movie...done
Liverpool [3] - 0 Bournemouth
Generating movie...done
Liverpool [1] - 0 Wolves
Generating movie...done
Liverpool [2] - 1 Chelsea
Generating movie...done
Liverpool [3] - 0 Norwich
Generating movie...done
Liverpool [2] - 0 Porto
Generating movie...done
Liverpool [2] - 0 Everton
Generating movie...done
Liverpool [2] - 1 Newcastle
Generating movie...done
Liverpool [2] - 0 Salzburg
Generating movie...done
Liverpool [2] - 0 Man City
Generating movie...done
Liverpool [1] - 0 Watford
Generating movie...done
Leicester 0 - [3] Liverpool
Generating movie...done
Liverpool [4] - 0 Barcelona
Generating movie...done
Porto 0 - [2] Liverpool
Generating movie...done


# generate movie (pitch_control)

In [4]:
fps = 20
field_dimen = (106., 68.)
n_grid_cells_x = 50
n_grid_cells_y = int(n_grid_cells_x * field_dimen[1] / field_dimen[0])
annotate=True
include_player_velocities=True
PlayerMarkerSize=10
PlayerAlpha=0.7
xgrid = np.linspace(-field_dimen[0] / 2.0, field_dimen[0] / 2.0, n_grid_cells_x)
ygrid = np.linspace(-field_dimen[1] / 2.0, field_dimen[1] / 2.0, n_grid_cells_y)
ppcf_a = np.zeros(shape=(len(ygrid), len(xgrid)))
ppcf_d = np.zeros(shape=(len(ygrid), len(xgrid)))
params = mpc.default_model_params(3)

def generate_pitch_control_movie(df_dict, team_color_dict, movie_dir, play):
    cmap = LinearSegmentedColormap.from_list("", [color for team, color in team_color_dict.items() if team]+[team_color_dict['Liverpool']])
    fname = os.path.join(movie_dir, f'{play}.mp4')

    fig, ax = mviz.plot_pitch(field_color='white', field_dimen=field_dimen)
    ffmpeg_writer = animation.writers['ffmpeg']
    writer = ffmpeg_writer(fps=fps, metadata={'title': play})
    writer.setup(fig=fig, outfile=fname, dpi=100)
    frame_range = df_dict[list(df_dict.keys())[0]].index

    with writer.saving(fig, fname, 100):
        for i in tqdm(frame_range):
            figobjs = []
            for team_name in df_dict.keys():
                team = df_dict[team_name].loc[i]
                color = team_color_dict[team_name]
                x_columns = [c for c in team.keys() if c[-2:].lower()=='_x' and c!='ball_x'] # column header for player x positions
                y_columns = [c for c in team.keys() if c[-2:].lower()=='_y' and c!='ball_y'] # column header for player y positions
                objs, = ax.plot( team[x_columns], team[y_columns], color=color, linestyle='None', marker='o', MarkerSize=PlayerMarkerSize, alpha=PlayerAlpha ) # plot player positions
                figobjs.append(objs)
                if include_player_velocities:
                    vx_columns = ['{}_vx'.format(c[:-2]) for c in x_columns] # column header for player x positions
                    vy_columns = ['{}_vy'.format(c[:-2]) for c in y_columns] # column header for player y positions
                    objs = ax.quiver( team[x_columns], team[y_columns], team[vx_columns], team[vy_columns], color=color, scale_units='inches', scale=10.,width=0.0015,headlength=5,headwidth=3,alpha=PlayerAlpha)
                    figobjs.append(objs)
                if annotate:
                    figobjs += [ax.text( team[x]+0.5, team[y]+0.5, x.split('_')[1], fontsize=10, color=color) for x,y in zip(x_columns,y_columns) if not (np.isnan(team[x]) or np.isnan(team[y]))]
            # plot ball
            objs, = ax.plot( team['ball_x'], team['ball_y'], color='yellow', marker='o', MarkerSize=6, alpha=1.0, LineWidth=0)
            figobjs.append(objs)
            for team, df in df_dict.items():
                if team == 'Liverpool':
                    attacking_players = mpc.initialise_players(df.loc[i], team, params)
                else:
                    defending_players = mpc.initialise_players(df.loc[i], team, params)

            for y in range(len(ygrid)):
                for x in range(len(xgrid)):
                    target_pos = np.array([xgrid[x], ygrid[y]])
                    ball_start_pos = np.array([df.loc[i]['ball_x'],
                                                df.loc[i]['ball_y']])
                    ppcf_a[y, x], ppcf_d[y, x] = mpc.calculate_pitch_control_at_target(target_pos,
                                                                                        attacking_players,
                                                                                        defending_players,
                                                                                        ball_start_pos,
                                                                                        params)
            
            objs = ax.imshow(np.flipud(ppcf_a),
                extent=(np.amin(xgrid), np.amax(xgrid), np.amin(ygrid), np.amax(ygrid)),
                interpolation='hanning',
                # interpolation="lanczos",
                vmin=0.0,
                vmax=1.0,
                cmap=cmap,
                alpha=0.5)
            figobjs.append(objs)
            # include match time at the top
            frame_minute =  int( df.loc[i]['Time [s]']/60. )
            frame_second =  ( df.loc[i]['Time [s]']/60. - frame_minute ) * 60.
            timestring = "%d:%1.2f" % ( frame_minute, frame_second  )  + " Attacking team control: {:.1f}%".format(ppcf_a.mean() * 100)
            objs = ax.text(-10,field_dimen[1]/2.+1., timestring, fontsize=14, color='black')
            figobjs.append(objs)
            # set legend
            figobjs.append(ax.legend(handles=[mpatches.Patch(color=color, label=team_name) for team_name, color in team_color_dict.items()], fontsize=12, title='Team'))
            writer.grab_frame()
            # Delete all axis objects (other than pitch lines) in preperation for next frame
            for figobj in figobjs:
                figobj.remove()
    plt.clf()
    plt.close(fig)

In [6]:
out_dir = os.path.join(base_dir, 'reports', 'movie', 'pitch_control')

for play in play_list:
    print('\n' + play)
    data_dir_tmp = os.path.join(data_dir, play)
    df_dict, color_dict = {}, {}
    for infile in os.listdir(data_dir_tmp):
        if not infile.startswith('events.'):
            team, color = infile.replace('.csv','').split('_')
            df = pd.read_csv(os.path.join(data_dir_tmp, infile), index_col=[0])
            df.columns = [c if c in ['Time [s]', 'ball_x', 'ball_y'] else f'{team}_{c}' for c in df.columns]
            df_dict[team] = df
            
            color_dict[team] = 'black' if color in ['white','lightgray'] else color

    generate_pitch_control_movie(df_dict, color_dict, out_dir, play)


Fulham 0 - [1] Liverpool
100%|██████████| 183/183 [21:21<00:00,  7.00s/it]
